In [40]:
%run "utils.ipynb"
import sqlite3
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt 

from contextlib import closing 
import math

In [4]:
db_name = 'macrodata.db'
excluded = [8, 62, 60] + [i for i in range(81, 89)]
selected =(slice(None), [i for i in range(1, 89) if i not in [8, 62, 60] + [i for i in range(81, 89)]])

def clean_output_table(table_name, selected, excluded_sectors):
    with closing(sqlite3.connect(db_name)) as engine:
        df = pd.read_sql('SELECT * FROM %s' %(table_name), engine).set_index(['year', 'sector_id'])
        df = df.loc[selected,:]
        df = df.drop(columns=[str(x) for x in excluded])
        df.reset_index().set_index(['year']).to_sql('filtered_%s' %(table_name), engine, index=True, if_exists='replace', index_label=['year'])
        
with closing(sqlite3.connect(db_name)) as engine:
    df = pd.read_sql('SELECT * FROM klems_nominal', engine).set_index(['year', 'sector_id'])
    excluded_sectors = df.columns[[3+x for x in excluded]].values
    included_sectors = df.columns[4:].difference(excluded_sectors).values
    aggregates = np.concatenate((['year', 'sector_id'], df.columns[:4].values))
    clean_output_table('klems_nominal', selected, excluded_sectors)
    clean_output_table('klems_real', selected, excluded_sectors)

In [5]:
with closing(sqlite3.connect(db_name)) as engine:
    df = pd.read_sql('SELECT * FROM filtered_klems_nominal', engine).set_index(['year', 'sector_id'])

In [6]:

def get_aggregated_results(df, selected):
    capital = df[['capital']].loc[selected,:].reset_index()
    labor = df[['labor']].loc[selected,:].reset_index()
    vadded = df[['capital', 'labor']].loc[selected,:].reset_index()
    output = df[['output']].loc[selected,:].reset_index()
    vadded_by_sector = pd.pivot_table(vadded,index=['year'], columns=['sector_id']).groupby(level=1, axis=1).sum()
    vadded_aggregate = pd.pivot_table(vadded,index=['year'], columns=['sector_id']).groupby(level=0, axis=1).sum()
    output_by_sector = pd.pivot_table(output,index=['year'], columns=['sector_id']).groupby(level=1, axis=1).sum()
    output_aggregate = pd.pivot_table(output,index=['year'], columns=['sector_id']).groupby(level=0, axis=1).sum()
    domar_weights = output_by_sector.apply(lambda x: x / vadded_aggregate.sum(axis=1).values)
    vadded_weights = vadded_by_sector.apply(lambda x: x / vadded_aggregate.sum(axis=1).values)
    capital_share = capital / output 
    labor_share = labor / output 
    scope = locals()
    return({item: eval(item, scope).reset_index().set_index(['year']) for item in 
            ['vadded_by_sector', 'vadded_aggregate', 'output_by_sector', 'output_aggregate', 'domar_weights', 'vadded_weights', 'capital_share', 'labor_share']})

In [7]:
with closing(sqlite3.connect(db_name)) as engine:
    nominal_df = pd.read_sql('SELECT * FROM klems_nominal', engine).set_index(['year', 'sector_id'])
    sectors = nominal_df.columns[4:]
    results = get_aggregated_results(nominal_df, selected)
    for table_name, rows in results.items():
        print('Writing [%s] into the database.' %(table_name))
        rows.to_sql(table_name, engine, index=True, if_exists='replace', index_label=['year'])

Writing [vadded_by_sector] into the database.
Writing [vadded_aggregate] into the database.
Writing [output_by_sector] into the database.
Writing [output_aggregate] into the database.
Writing [domar_weights] into the database.
Writing [vadded_weights] into the database.
Writing [capital_share] into the database.
Writing [labor_share] into the database.


In [8]:
with closing(sqlite3.connect(db_name)) as engine:
    real_df = pd.read_sql('SELECT * FROM filtered_klems_real', engine).set_index(['year', 'sector_id']).loc[selected,:].reset_index()
    real_df.sort_values(['sector_id', 'year'], inplace=True, kind='mergesort')
    nominal_df = pd.read_sql('SELECT * FROM filtered_klems_nominal', engine).set_index(['year', 'sector_id']).loc[selected,:].reset_index()
    nominal_df.sort_values(['sector_id', 'year'], inplace=True, kind='mergesort')
    real_df['year'], nominal_df['year'] = pd.to_datetime(real_df['year'], format='%Y'), pd.to_datetime(nominal_df['year'], format='%Y')    

In [9]:
aggregates = ['output', 'capital', 'labor']

def get_growth_rates(in_df, log):
    df = in_df.copy(); df.iloc[:,[i for i in range(3,84)]] = df.iloc[:,[i for i in range(3,84)]].applymap(log)
    df = df.set_index(['year', 'sector_id'])
    return(df.groupby('sector_id', sort=False).diff().reset_index().set_index(['year', 'sector_id']))

def get_rollavg_shares(in_df):
    df = in_df.copy().drop(columns='sector')
    df['inputs'] = df['output'] - df['capital'] - df['labor']
    df[[x for x in df.columns.values if x.isdigit()]] = df[[x for x in df.columns.values if x.isdigit()]].apply(lambda x: x / df.inputs.values)
    df[aggregates] = df[aggregates].apply(lambda x: x / df.output.values)
    df = df.set_index(['year', 'sector_id'])
    return(df.reset_index().set_index(['year']).groupby('sector_id').rolling(2).mean().drop(columns=['inputs', 'sector_id']))


In [10]:
log_diffs = get_growth_rates(real_df, log=lambda x: math.log(x) if x > 0 else 0).fillna(0)
avg_share = get_rollavg_shares(nominal_df).fillna(0)
avg_share.index = log_diffs.index
tornqvist = (avg_share[[x for x in avg_share.columns if x.isdigit()]] * log_diffs[[x for x in log_diffs.columns if x.isdigit()]]).sum(axis=1)
tornqvist[tornqvist.index.get_level_values(0).year == 1960] = 0

In [11]:
ndf = nominal_df.copy().set_index(['year', 'sector_id']); xi = pd.DataFrame(ndf['output'] - ndf['capital'] - ndf['labor'])

In [12]:
base_year = 1996; base_ix = 46 - 1 - (2005 - base_year)
levels = tornqvist.fillna(0).apply(math.exp).groupby('sector_id').cumprod()
levels = levels.groupby('year').apply(lambda x: x / levels.groupby('sector_id').nth(base_ix))
rebased_levels = levels.groupby('year').apply(lambda x: x * xi.groupby('sector_id').nth(base_ix).values.flatten()).unstack('sector_id')
rebased_prices = xi.unstack('sector_id') / rebased_levels 

In [13]:
ldiffs = log_diffs[log_diffs.index.get_level_values(0).year > 1960]
shares = avg_share[avg_share.index.get_level_values(0).year > 1960]
tqvist = tornqvist[tornqvist.index.get_level_values(0).year > 1960]
sectoral_tfp = (ldiffs.output.unstack() - (ldiffs.capital * shares.capital).unstack() - (ldiffs.labor * shares.labor).unstack() - (1 - (shares.capital + shares.labor)).unstack() * tqvist.unstack())
sectoral_tfp_vadd = sectoral_tfp / (shares.capital + shares.labor).unstack()

In [14]:
input_share = pd.DataFrame(((ndf.output - ndf.capital - ndf.labor) / ndf.output).unstack('sector_id'))
avg_input_share = input_share.rolling(2).mean().stack()
avg_input_share.index = tqvist.index

In [15]:
with closing(sqlite3.connect(db_name)) as engine:
    domar_weights = pd.read_sql('SELECT * FROM domar_weights', engine)
    value_weights = pd.read_sql('SELECT * FROM vadded_weights', engine)
    domar_weights['year'], value_weights['year'] = pd.to_datetime(domar_weights['year'], format='%Y'), pd.to_datetime(value_weights['year'], format='%Y')    
    domar_weights = domar_weights.set_index('year')
    value_weights = value_weights.set_index('year')

In [16]:
avg_domar_weights = domar_weights.rolling(2).mean()[domar_weights.index.year > 1960]
avg_value_weights = value_weights.rolling(2).mean()[value_weights.index.year > 1960]
avg_value_share = (shares.capital + shares.labor).unstack()
aggregate_tfp_growth = pd.DataFrame(np.sum((avg_domar_weights.values * sectoral_tfp.values), axis=1), index=pd.date_range(start='1961-01-01', end='2005-01-01', freq='AS'))

In [17]:
tfp_covars = np.cov(np.vstack([np.zeros((77)), sectoral_tfp.values]), rowvar=False)
fundamental_vol_diag = domar_weights.apply(lambda x: (x*tfp_covars.diagonal())**2, axis=1).sum(axis=1)
fundamental_vol_full = pd.Series(np.apply_along_axis(lambda x: (np.multiply.outer(x, x) * tfp_covars).sum(), 1, domar_weights.values), index=fundamental_vol_diag.index)
fundamental_vol_df = pd.DataFrame({'inner': fundamental_vol_diag, 'outer': fundamental_vol_full}).applymap(np.sqrt)
fundamental_vol_df = fundamental_vol_df.sub(fundamental_vol_df.mean(axis=0), axis=1)

In [18]:
fig, ax = plt.subplots(figsize=(15,15))
fundamental_vol_df.inner.plot(ax=ax, style='b-')
fundamental_vol_df.outer.plot(ax=ax, style='g-', secondary_y=True)
ax.legend([ax.get_lines()[0], ax.right_ax.get_lines()[0]], ['Fundamental vol (inner product)', 'Fundamental vol (incl. outer product)'], loc='best')

In [30]:
sectoral_tfp.to_csv('sectoral_tfp.csv')

In [25]:
aggregate_tfp_growth.to_csv('aggregate_tfp_growth.csv')

In [27]:
avg_value_weights.to_csv('avg_value_weights.csv')

In [33]:
avg_domar_weights.to_csv('avg_domar_weights.csv')

In [32]:
fundamental_vol_df.to_csv('fundamental_vol.csv')

In [35]:
avg_input_share.to_csv('avg_input_share.csv')

In [37]:
real_df.to_csv('real_io.csv')

In [40]:
nominal_df.to_csv('nominal_io.csv')

In [42]:
rebased_levels.to_csv('rebased_io.csv')

In [43]:
rebased_prices.to_csv('rebased_prices_io.csv')

In [61]:
sector_index = real_df[['sector_id', 'sector']].drop_duplicates()
sector_index.set_index(['sector_id'])
sector_index.to_csv('sector_index.csv')

In [94]:
with closing(sqlite3.connect(db_name)) as engine:
    data=pd.read_sql('SELECT * FROM vadded_weights', engine)

In [80]:
pd.DataFrame(input_share.unstack(), columns=['input_share']).reset_index().set_index(['year', 'sector_id']).sort_index(level=0).to_csv('input_shares.csv')

In [84]:
np.savetxt('sector_varcov.csv', tfp_covars)

In [86]:
domar_weights.to_csv('domar_weights.csv')

In [19]:
sectoral_tfp

sector_id,1,2,3,4,5,6,7,9,10,11,...,71,72,73,74,75,76,77,78,79,80
year,,,,,,,,,,,,,,,,,,,,,
1961-01-01,0.012523,0.079934,0.029166,-0.014004,0.025822,0.022692,0.008177,-0.000336,0.031041,0.007274,...,0.033603,0.110485,-0.007223,-0.080057,0.032858,-0.025882,-0.028692,0.012836,0.019686,0.000000
1962-01-01,-0.011890,0.039045,0.004204,-0.004085,0.004604,0.041508,0.011923,0.001012,-0.000670,0.016080,...,0.035043,0.136601,-0.001185,-0.047345,0.003396,-0.024701,-0.021843,-0.005534,0.016476,0.000000
1963-01-01,0.017869,0.073516,-0.031878,-0.010092,0.005204,0.049291,0.015730,-0.002295,0.013942,0.027012,...,0.078262,-0.076333,-0.041876,-0.134486,0.001066,-0.027455,-0.013799,-0.005219,0.017065,0.000000
1964-01-01,0.020374,-0.033429,-0.029905,0.011140,0.026169,-0.098895,0.017561,-0.005541,0.002839,0.004143,...,-0.051837,0.145749,0.053508,0.107136,-0.009896,-0.004590,0.026806,0.002936,0.041776,0.000000
1965-01-01,0.009471,0.026474,-0.003027,-0.024994,0.014222,0.175946,0.014683,-0.004962,-0.001270,0.005514,...,0.013089,0.070913,-0.011142,-0.074821,-0.028076,-0.004207,0.023725,0.003083,-0.002628,0.000000
1966-01-01,-0.015657,0.010093,0.012674,0.008630,0.003603,0.037100,0.008972,-0.021190,-0.015787,-0.010234,...,-0.027418,0.028508,-0.001640,-0.025196,0.001383,0.025461,0.006514,0.007687,0.012272,0.000000
1967-01-01,0.051508,-0.024007,-0.030290,-0.050773,-0.026403,0.028071,0.008851,-0.013183,0.031920,-0.005997,...,0.010448,-0.110768,0.010171,-0.004736,-0.007991,-0.006128,0.001855,-0.025915,-0.004348,0.000000
1968-01-01,0.013087,0.011216,-0.002178,0.010531,-0.001813,0.021799,0.012453,0.011813,-0.029616,0.013428,...,0.027475,0.055138,0.003877,-0.019981,-0.019022,-0.004423,0.017387,-0.066912,-0.076779,0.000000
1969-01-01,-0.006305,-0.019101,-0.023586,-0.017399,0.010981,0.002806,0.008087,-0.040847,-0.043921,0.015401,...,0.031133,0.006962,-0.000840,-0.057778,-0.017018,0.009385,-0.035566,-0.029470,-0.050054,0.000000


In [26]:
238979.594000-37220.000000-54626.777000

147132.817

In [29]:
rebased_levels

sector_id,1,2,3,4,5,6,7,9,10,11,...,71,72,73,74,75,76,77,78,79,80
year,,,,,,,,,,,,,,,,,,,,,
1960-01-01,103583.333853,2918.244596,2966.730346,4340.903986,4861.849928,3575.481343,60935.423578,204450.163649,26369.660636,11208.450989,...,26759.437424,640.599605,7253.429083,4836.083268,9885.347264,9303.667576,18278.356213,3329.505387,16454.005787,0.000000
1961-01-01,104876.782021,2911.963279,3011.095765,4147.962530,4831.211487,3265.756485,62217.354307,207758.635699,25413.846076,11031.549168,...,26419.358449,725.428545,7900.118893,4785.160200,11090.717150,9904.658686,19209.364584,3588.382572,17488.483342,0.000000
1962-01-01,106056.239732,2926.481935,3076.499455,4025.541999,5022.136152,3270.857968,64169.154788,215600.260854,26751.466656,11882.017981,...,26652.843584,820.034008,8510.845423,4647.762086,11707.088788,10580.643718,20072.471216,3933.262156,18999.823279,0.000000
1963-01-01,109436.396295,3015.362906,3212.000117,4114.262555,5241.441786,3411.369913,63028.273782,225131.942159,29463.812489,12603.827920,...,27134.303923,1037.082002,9281.618031,5032.862843,12670.779012,11356.613914,21219.518812,4265.786970,20048.323691,0.000000
1964-01-01,107274.884846,3076.445511,3335.007539,4512.390740,5526.305283,3432.393586,63156.826447,230147.337037,31633.900280,13288.042555,...,28823.253323,1376.389074,10025.255296,5471.171486,13530.453763,12160.304064,23372.541226,4395.591768,21573.787902,0.000000
1965-01-01,111378.658541,3060.945272,3381.937656,4701.676010,5904.248161,3591.322325,62734.953930,241424.921361,32516.469586,14208.545114,...,28115.375712,1485.833493,10824.051066,5432.458060,14669.542719,13513.539295,24992.965925,5020.281201,23512.077199,0.000000
1966-01-01,114310.184106,2991.150230,3375.493963,4896.969601,6202.854096,3659.458527,64759.626555,247820.497023,33321.986778,15075.308994,...,28407.556115,1656.997634,11745.912564,6462.390492,16004.463540,14926.424604,27003.339132,5577.474382,26269.686231,0.000000
1967-01-01,114258.589475,3101.885456,3571.867707,4088.587545,6108.703993,3725.283597,67723.643509,251728.221520,34997.413558,15467.615506,...,31642.713737,2048.056483,13710.070022,7503.890313,16904.659497,16213.984204,29595.106271,5977.848085,28050.676335,0.000000
1968-01-01,114762.440227,3236.975065,3619.162678,4619.166282,6128.366030,3452.328916,71068.149075,267126.279258,38514.643256,16349.108146,...,32224.942760,2921.256153,15562.163030,8239.363867,17432.082197,18030.588250,31976.342273,6391.142530,29018.273160,0.000000


In [30]:
input_share

sector_id,1,2,3,4,5,6,7,9,10,11,...,71,72,73,74,75,76,77,78,79,80
year,,,,,,,,,,,,,,,,,,,,,
1960-01-01,0.639241,0.405401,0.512647,0.488744,0.434128,0.332545,0.426090,0.546190,0.622997,0.526955,...,0.520875,0.328034,0.315014,0.439098,0.401541,0.414786,0.425123,0.418360,0.394147,0.000000
1961-01-01,0.629414,0.414376,0.518469,0.486984,0.428428,0.319707,0.423893,0.539081,0.610505,0.521227,...,0.508809,0.326510,0.312847,0.441439,0.405162,0.419186,0.428908,0.420457,0.396536,0.000000
1962-01-01,0.627780,0.460029,0.564618,0.482600,0.425837,0.312636,0.421119,0.537126,0.615124,0.522919,...,0.482996,0.326411,0.312126,0.446459,0.409956,0.421697,0.432772,0.423627,0.399247,0.000000
1963-01-01,0.628780,0.441677,0.544733,0.481118,0.421549,0.306090,0.409515,0.533955,0.619818,0.522335,...,0.479945,0.327015,0.311596,0.452719,0.415561,0.418488,0.437530,0.426564,0.402249,0.000000
1964-01-01,0.621009,0.414036,0.525678,0.481627,0.418638,0.298758,0.399857,0.533340,0.621423,0.524760,...,0.452641,0.324313,0.308096,0.455020,0.416704,0.411949,0.438226,0.426200,0.400960,0.000000
1965-01-01,0.618580,0.404610,0.522823,0.479285,0.416164,0.290589,0.390702,0.533314,0.618549,0.526985,...,0.425957,0.324945,0.308194,0.462470,0.422234,0.416892,0.443037,0.430943,0.405704,0.000000
1966-01-01,0.616948,0.399007,0.525381,0.478808,0.415684,0.284179,0.388155,0.535600,0.620780,0.530261,...,0.416363,0.325308,0.308007,0.468883,0.427017,0.425938,0.447396,0.435862,0.410485,0.000000
1967-01-01,0.613296,0.383589,0.524654,0.474661,0.416144,0.286083,0.391074,0.538096,0.624056,0.531833,...,0.435466,0.329712,0.311165,0.478993,0.436926,0.435321,0.456610,0.443265,0.417926,0.000000
1968-01-01,0.612122,0.370656,0.522550,0.484606,0.414191,0.273621,0.394015,0.540832,0.631607,0.539314,...,0.428318,0.332399,0.312395,0.487135,0.444088,0.440526,0.463390,0.446762,0.420990,0.000000


In [38]:
log_gdp = pd.DataFrame(rebased_levels.values / input_share.values, index=input_share.index, columns=input_share.columns).fillna(1.0).applymap(np.log)

In [47]:
lambda_ravn_uhlig = 6.25*(1600/4**4)
log_sector_output_cycle = log_gdp.apply(lambda x: sm.tsa.filters.hpfilter(x, lambda_ravn_uhlig)[0])
log_sector_output_trend = log_gdp.apply(lambda x: sm.tsa.filters.hpfilter(x, lambda_ravn_uhlig)[1])

In [49]:
log_sector_output_cycle.to_csv('log_sector_output_cycle.csv')